In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from pytorch_lightning.metrics.functional import auroc
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import timm
import random
warnings.filterwarnings('ignore')

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
GENERAL = '../input/vinbigdata-competition-jpg-data-2x-downsampled'
TRAIN_DIR = '../input/vinbigdata-competition-jpg-data-2x-downsampled/train/train'
TEST_DIR = '../input/vinbigdata-competition-jpg-data-2x-downsampled/test/test'

In [6]:
df = pd.read_csv(os.path.join(GENERAL, 'train_downsampled.csv'))

In [7]:
df_cl = df.copy()
df_cl['label'] = (df_cl['class_id'] != 14).astype(int)
df_cl = df_cl[['image_id', 'label']].drop_duplicates()
df_cl['image_id'] = df_cl['image_id'].map(lambda x: x + '.jpg')

In [8]:
model_names = timm.list_models(pretrained = True)

In [9]:
# list of available models pretrained models in timm
model_names

['adv_inception_v3',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnetlight',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_lite0',
 'ens_adv_inception_resnet_v2',
 'ese_vovnet19b_dw',
 'ese_vovnet39b',
 'fbnetc_100',
 'gluon_inception_v3',
 'gluon_resnet18_v1b',
 'gluon_resnet34_v1b',
 'gluon_resnet50_v1b',
 'gluon_resnet50_v1c',
 'gluon_resnet50_v1d',
 'gluon_resnet50_v1s',
 'gluon_resn

In [10]:
CFG = {
    'seed':42,
    'img_size':256,
    'model_arch':'tv_resnet50',
    'pretrained':True,
    'train_transforms': A.Compose([A.Resize(256, 256), ToTensorV2(p=1)], p = 1),
    'valid_transforms': A.Compose([A.Resize(256, 256), ToTensorV2(p=1)], p = 1),
    'n_class':2,
    'TRAIN_DIR':TRAIN_DIR,
    'VAL_DIR':TRAIN_DIR,
    'use_label':True,
    'train_batch_size':8,
    'valid_batch_size':8,
    'min_epochs':1,
    'max_epochs':30,
    'save_top_k':3,
    'val_size':0.1,
    'optim':torch.optim.Adam,
    'learning_rate':1e-2,
    'loss_fn':F.cross_entropy
}

In [11]:
class ClassifierDataset(Dataset):
  
  def __init__(self, data, image_dir, transforms = None, use_label = True):
    super(ClassifierDataset, self).__init__()
    self.data = data
    self.use_label = use_label
    self.image_dir = image_dir
    self.transforms = transforms

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, idx):
    img_id = self.data.iloc[idx, 0]
    img_path = os.path.join(self.image_dir, img_id)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0

    if self.transforms is not None:
      img = self.transforms(image = img)['image']

    if self.use_label:
      label = self.data.iloc[idx, 1]
      return img, label
    else:
      return img

In [12]:
class ClassifierModel(pl.LightningModule):
  def __init__(self, cfg, train, valid):
    super(ClassifierModel, self).__init__()
    
    self.cfg = cfg
    self.train_df = train_df
    self.optim = cfg['optim']
    self.lr = cfg['learning_rate']
    self.loss_fn = cfg['loss_fn']
    self.valid_df = valid
    self.model = timm.create_model(cfg['model_arch'], pretrained=cfg['pretrained'])
    n_features = self.model.fc.in_features
    self.model.fc = nn.Linear(n_features, cfg['n_class'])
    self.save_hyperparameters()
  
  def forward(self, x):
    x = x.float()
    return self.model(x)

  def training_step(self, batch, batch_idx):
    x, y = batch
    x = x.float()
    logits = self.model(x)
    loss = self.cfg['loss_fn'](logits, y)
    self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    x = x.float()
    logits = self.model(x)
    loss = self.cfg['loss_fn'](logits, y)
    self.log('val_loss', loss, on_step = False, on_epoch = True, prog_bar = True, logger = True)
    return loss

  def train_dataloader(self):
      train_dataset = ClassifierDataset(self.train_df, self.cfg['TRAIN_DIR'], transforms = self.cfg['train_transforms'], use_label = self.cfg['use_label'])
      return DataLoader(train_dataset, batch_size=self.cfg['train_batch_size'])

  def val_dataloader(self):
      val_dataset = ClassifierDataset(self.valid_df, self.cfg['VAL_DIR'], transforms = self.cfg['valid_transforms'], use_label = self.cfg['use_label'])
      return DataLoader(val_dataset, batch_size=self.cfg['valid_batch_size'])

  def test_dataloader(self):
      test_dataset = ClassifierDataset(self.valid_df, self.cfg['VAL_DIR'], transforms = self.cfg['valid_transforms'], use_label = self.cfg['use_label'])
      return DataLoader(val_dataset, batch_size=self.cfg['valid_batch_size'])

  

  def configure_optimizers(self):
    return self.cfg['optim'](self.parameters(), lr=self.cfg['learning_rate'])

In [13]:
os.makedirs('/kaggle/working/checkpoints', exist_ok = True)

In [14]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='/kaggle/working/checkpoints',
    filename='classifier_model-{epoch:02d}-{val_auc:.2f}',
    save_top_k=CFG['save_top_k'],
    mode='min'
)

In [15]:
seed_everything(CFG['seed'])

In [16]:
train_df, valid_df = train_test_split(df_cl, test_size = CFG['val_size'], stratify = df_cl['label'], random_state = CFG['seed'])

In [17]:
cl_model = ClassifierModel(cfg = CFG, train = train_df, valid = valid_df)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [18]:
trainer = pl.Trainer(gpus=1, checkpoint_callback=checkpoint_callback, min_epochs = CFG['min_epochs'], max_epochs = CFG['max_epochs'],
 deterministic = True, 
 logger = True)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [19]:
trainer.fit(cl_model)


  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 23.5 M
---------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

...

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

1

In [20]:
torch.save(cl_model.model.state_dict(), '/kaggle/working/classifier_resnet50.pth')

In [ ]:
cl_model.eval();

In [22]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [23]:
valid_df['prediction'] = 0

In [24]:
for img in valid_df['image_id'].unique():
    img_path = os.path.join(CFG['VAL_DIR'], img)
    im = cv2.imread(img_path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im / 255.0
    im = CFG['valid_transforms'](image = im)['image']
    prediction = cl_model(im.unsqueeze_(0)).detach().numpy()[0]
    prediction = softmax(prediction)
    prediction = prediction[1]
    valid_df.loc[valid_df['image_id'] == img, 'prediction'] = prediction
    

In [25]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [26]:
roc_auc_score(valid_df['label'], valid_df['prediction'])

0.9529068506738174

In [27]:
y_true = valid_df['label'].values

In [28]:
threshold = 0.5
y_prediction = np.array(valid_df['prediction'] > threshold).astype(int)
print(classification_report(y_true, y_prediction))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      1061
           1       0.88      0.73      0.80       439

    accuracy                           0.89      1500
   macro avg       0.89      0.84      0.86      1500
weighted avg       0.89      0.89      0.89      1500



In [29]:
threshold = 0.6
y_prediction = np.array(valid_df['prediction'] > threshold).astype(int)
print(classification_report(y_true, y_prediction))

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1061
           1       0.90      0.72      0.80       439

    accuracy                           0.89      1500
   macro avg       0.90      0.84      0.86      1500
weighted avg       0.90      0.89      0.89      1500



In [30]:
threshold = 0.7
y_prediction = np.array(valid_df['prediction'] > threshold).astype(int)
print(classification_report(y_true, y_prediction))

              precision    recall  f1-score   support

           0       0.88      0.97      0.93      1061
           1       0.91      0.69      0.79       439

    accuracy                           0.89      1500
   macro avg       0.90      0.83      0.86      1500
weighted avg       0.89      0.89      0.89      1500



In [31]:
threshold = 0.8
y_prediction = np.array(valid_df['prediction'] > threshold).astype(int)
print(classification_report(y_true, y_prediction))

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      1061
           1       0.92      0.65      0.76       439

    accuracy                           0.88      1500
   macro avg       0.89      0.81      0.84      1500
weighted avg       0.89      0.88      0.87      1500



In [32]:
threshold = 0.9
y_prediction = np.array(valid_df['prediction'] > threshold).astype(int)
print(classification_report(y_true, y_prediction))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1061
           1       0.95      0.61      0.74       439

    accuracy                           0.88      1500
   macro avg       0.90      0.80      0.83      1500
weighted avg       0.89      0.88      0.87      1500



In [33]:
threshold = 0.95
y_prediction = np.array(valid_df['prediction'] > threshold).astype(int)
print(classification_report(y_true, y_prediction))

              precision    recall  f1-score   support

           0       0.84      0.99      0.91      1061
           1       0.96      0.55      0.70       439

    accuracy                           0.86      1500
   macro avg       0.90      0.77      0.80      1500
weighted avg       0.88      0.86      0.85      1500

